# Playground 

this is a general playground to try out stuff. probably this shouldn't even be in the repo :-P


In [1]:
import mb_modelbase as mbase; import pandas as pd; 
import numpy as np
import multiprocessing as mp
import os

In [2]:
mymod = mbase.MixableCondGaussianModel("Artifical_Dataset")
dataset = pd.read_csv("doc/artificial_dataset.csv", index_col=0)
mymod.fit(df=dataset, fit_algo="map")

In [3]:
mymod.loglikelihood()

-31089.836379679939

# Tests for Parallel Envirnment

In [12]:
num_samp = 1000
cpus = mp.cpu_count()
print("Samples:", num_samp, ", CPUs:", cpus)

Samples: 1000 , CPUs: 4


In [25]:
%%timeit
result1 = mymod._sample(num_samp)

In [14]:
%%timeit
output = mp.Queue()
def samples(self, k):
    a = self._sample(k)
    output.put(a)
        
processes = [mp.Process(target=samples, args=(mymod, (int)(num_samp/cpus))) for _ in range(cpus)]
[process.start() for process in processes]
[process.join() for process in processes]
result2 = [output.get() for p in processes]

589 ms ± 23.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
with mp.Pool(cpus) as p:
    result3 = p.map(mymod._sample, cpus*[(int)(num_samp/cpus)])

667 ms ± 44.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
